In [1]:
import pandas as pd
import numpy as np
import json
import os
import sys
import time
import datetime
import tensorflow as tf
from pandas import read_csv
from numpy import array
from numpy import hstack
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils

from sklearn.metrics import mean_squared_error, r2_score

from pandas import read_csv
from datetime import datetime
from math import sqrt
from matplotlib import pyplot
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Bidirectional
from keras import optimizers

import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import chart_studio.plotly as py
import cufflinks as cf 
cf.go_offline(connected=True)

import seaborn as sns    
import matplotlib as mpl
import matplotlib.font_manager as fm

# 데이터 임포트

In [2]:
data = pd.read_csv('올림픽.csv',
                        skiprows=1, 
                        names=['INDATES','INYEAR', 'INMON', 'INDAY', 'INHOUR', 'OUTDATES', 'OUTYEAR', 'OUTMON', 'OUTDAY', 'OUTHOUR', 'INDATE', 'OUTDATE', 'cnt'],
                        engine='python',
                        encoding ='utf-8-sig')

rawdata = data.copy()

#INDATE, OUTDATE datetime 형식으로 바꾸기
rawdata['INDATE'] = pd.to_datetime(rawdata['INDATE'])
rawdata['OUTDATE'] = pd.to_datetime(rawdata['OUTDATE'])

# 이용시간 구하기
using_time = []
for i in range(len(rawdata.index)):
    using_time.append(rawdata['OUTDATE'][i] - rawdata['INDATE'][i])
    
rawdata['USING'] = list(using_time)

In [3]:
#rawdata

# 시간별

In [4]:
hour = pd.DataFrame(rawdata.groupby('INHOUR')['USING'].mean(numeric_only=False))
hour

,USING
INHOUR,
0,0 days 08:13:42.909430170
1,0 days 08:36:09.466893039
2,0 days 11:20:15.529942370
3,0 days 06:13:17.780713342
4,0 days 05:02:36.111568207
5,0 days 03:40:22.396360081
6,0 days 04:06:10.240858481
7,0 days 05:26:19.527880075
8,0 days 05:46:17.307528191


In [5]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14,
        
    }
}
layout['title'] = "올림픽공원 입차 시간별 평균 이용시간"

bar = hour.iplot(kind='bar', layout=layout, xTitle='Hour', yTitle = '시간', color='skyblue', asFigure=True)
bar

In [6]:
bar.write_image("/home/datastory/no_13.올림픽 공원 차량 이용 현황 데이터를 이용한 차량 이용객, 평균 이용 시간 분석/13/img_4_1.png")
                
bar_json = bar.to_json()
bar_json
json_obj = json.loads(bar_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./13/img_4_1.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)

# 일별

In [7]:
day = pd.DataFrame(rawdata.groupby('INDAY')['USING'].mean(numeric_only=False))
day

,USING
INDAY,
1,0 days 02:46:57.937278764
2,0 days 02:59:14.591892459
3,0 days 02:54:35.930419931
4,0 days 02:56:38.944388491
5,0 days 03:00:05.334360300
6,0 days 02:58:19.995116568
7,0 days 03:00:29.617040724
8,0 days 03:01:52.647300004
9,0 days 03:00:25.691088873


In [8]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14,
        
    }
}
layout['title'] = "올림픽공원 입차 일별 평균 이용시간"

bar = day.iplot(kind='bar', layout=layout, xTitle='Day', yTitle = '시간', color='darkslateblue', asFigure=True)
bar

In [9]:
bar.write_image("/home/datastory/no_13.올림픽 공원 차량 이용 현황 데이터를 이용한 차량 이용객, 평균 이용 시간 분석/13/img_5_1.png")
                
bar_json = bar.to_json()
bar_json
json_obj = json.loads(bar_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./13/img_5_1.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)

# 월별

In [10]:
mon = pd.DataFrame(rawdata.groupby('INMON')['USING'].mean(numeric_only=False))
mon

,USING
INMON,
1,0 days 03:00:59.642113845
2,0 days 02:59:18.492968741
3,0 days 02:58:52.787291053
4,0 days 02:57:50.280171765
5,0 days 03:02:16.342152508
6,0 days 03:01:20.326930380
7,0 days 03:00:25.641376500
8,0 days 03:00:30.519309312
9,0 days 02:56:41.255138865


In [11]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14,
        
    }
}
layout['title'] = "올림픽공원 입차 월별 평균 이용시간"

bar = mon.iplot(kind='bar', layout=layout, xTitle='MONTH', yTitle = '시간', color='blue', asFigure=True)
bar

In [12]:
bar.write_image("/home/datastory/no_13.올림픽 공원 차량 이용 현황 데이터를 이용한 차량 이용객, 평균 이용 시간 분석/13/img_4_2.png")
                
bar_json = bar.to_json()
bar_json
json_obj = json.loads(bar_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./13/img_4_2.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)

# 계절별

In [13]:
sea = pd.DataFrame(rawdata.groupby('INMON')['USING'].mean(numeric_only=False))

sea.reset_index(inplace=True)

se = ['겨울','봄','여름','가을']

#계절로 나누기
win = (sea['USING'][0] + sea['USING'][1] + sea['USING'][11]) / 3
spr = (sea['USING'][2] + sea['USING'][3] + sea['USING'][4]) / 3
summ = (sea['USING'][5] + sea['USING'][6] + sea['USING'][7]) / 3 
aut = (sea['USING'][8] + sea['USING'][9] + sea['USING'][10]) / 3 

season_list = []
season_list.append(win)
season_list.append(spr)
season_list.append(summ)
season_list.append(aut)

table = pd.DataFrame(season_list)
table.index = se

In [14]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14,
        
    }
}
layout['title'] = "올림픽공원 입차 계절별 평균 이용시간"

bar = table.iplot(kind='bar', layout=layout, xTitle='Season', yTitle = '시간', color='powderblue', asFigure=True)
bar

In [15]:
bar.write_image("/home/datastory/no_13.올림픽 공원 차량 이용 현황 데이터를 이용한 차량 이용객, 평균 이용 시간 분석/13/img_5_2.png")
                
bar_json = bar.to_json()
bar_json
json_obj = json.loads(bar_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./13/img_5_2.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)